In [1]:
import collections, math, numpy as np, pandas as pd, matplotlib, matplotlib.pyplot as plt, seaborn as sns
import numpy as np, scipy as sp, scipy.stats, scipy.stats.contingency, matplotlib, matplotlib.pyplot as plt, seaborn as sns
%load_ext autoreload
%autoreload 2
from af2genomics import *

In [2]:
# pockets for P30101-F1 above pocket score of 800
df_pockets = read_pockets().query('struct_id == "P30101-F1"').query('pocket_score_combined_scaled > 800').copy()# & pocket_id == 3').squeeze().pocket_cl_file
df_pockets

,struct_id,pocket_id,pocket_energy,pocket_n_points,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_score,pocket_cl_file,pocket_cl_isfile,...,pocket_pLDDT_mean90,pocket_score_recap,pocket_score_pLDDT_mean,pocket_score_pLDDT_median,pocket_score_pLDDT_q1,pocket_score_pLDDT_q3,pocket_score_energy,pocket_score_combined,pocket_is_ubs,pocket_score_combined_scaled
129896,P30101-F1,1,619.264999,1444.0,10.274899,0.428854,0.842742,99.811099,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,93.543953,99.811115,873394.150657,9476.065739,9107.263728,9683.423297,42.804384,3.265193e+13,False,998.990444
129897,P30101-F1,2,188.258000,462.0,6.745361,0.407485,0.808884,44.813572,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,92.543226,44.813542,383794.573174,4151.529310,4050.250637,4292.692062,18.260839,2.689790e+12,False,956.804331
129898,P30101-F1,3,152.893000,376.0,5.670105,0.406630,0.792614,41.660055,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,96.667586,41.660092,389297.475767,4066.437969,3972.286244,4086.434795,16.940256,2.567339e+12,False,954.290153
129899,P30101-F1,4,97.340000,214.0,3.778929,0.454860,0.847981,40.720888,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,96.361000,40.720892,378111.456784,3965.603678,3906.863797,3992.784871,18.522297,2.648101e+12,False,955.968837
129900,P30101-F1,5,130.501000,320.0,5.641483,0.407816,0.825397,38.644020,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,95.559524,38.644035,352882.606028,3716.395403,3625.581956,3756.198744,15.759641,1.962487e+12,False,935.606259
129901,P30101-F1,6,78.895000,195.0,3.892190,0.404590,0.825666,34.154605,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,90.425789,34.154614,279276.173448,3194.480206,3033.782789,3271.328067,13.818607,1.077785e+12,False,860.314463
129902,P30101-F1,7,95.442000,229.0,4.722937,0.416777,0.809717,31.789918,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,90.033333,31.789950,257689.110630,2964.091954,2751.099504,3059.461708,13.249329,8.798042e+11,False,818.480825
129903,P30101-F1,8,73.338000,176.0,3.893973,0.416693,0.826733,30.892274,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,94.516667,30.892302,275973.049133,2959.788772,2885.956237,3007.671797,12.872612,9.803926e+11,False,841.932502
129904,P30101-F1,9,83.208000,253.0,4.663325,0.328885,0.751479,30.637892,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,96.518889,30.637868,285419.412846,2979.994565,2942.922716,3001.058116,10.076347,8.208619e+11,False,803.250303
129906,P30101-F1,11,79.702000,150.0,4.196395,0.531347,0.893229,28.519418,results/23.04_bindfunc/af2.obabel_hxr.autosite...,True,...,95.806471,28.519409,261776.309975,2766.953934,2696.510972,2791.480634,15.153693,1.038435e+12,False,852.698652


In [3]:
#read_autosite_cl('/cluster/project/beltrao/jjaenes/22.12_pocketomes/' + pocket_file_path)

In [4]:
# center-of-mass for peptides defined by residue [219, 220, 221, 222, 223, 224, 225, 226]
pepcom_x, pepcom_y, pepcom_z = resid_center_of_mass('/cluster/project/beltrao/jjaenes/22.12_pocketomes/results/23.04_bindfunc/af2/P3/01/01/P30101-F1.pdb', [219, 220, 221, 222, 223, 224, 225, 226])
pepcom_x, pepcom_y, pepcom_z

(2.298625022172928, 20.522125005722046, -21.970125198364258)

In [5]:
def min_dist_autosite_cl(fp, x, y, z):
    df = read_autosite_cl(fp)
    df['dist_squared'] = (df['X'] - x)**2 + (df['Y'] - y)**2 + (df['Z'] - z)**2
    return math.sqrt(df['dist_squared'].min())

def closest_pocket_id(df_pocket_, pepcom_x, pepcom_y, pepcom_z):
    df_pocket_['min_dist'] = [* map(lambda cl_file: min_dist_autosite_cl('/cluster/project/beltrao/jjaenes/22.12_pocketomes/' + cl_file, pepcom_x, pepcom_y, pepcom_z), df_pocket_['pocket_cl_file']) ]
    return df_pocket_['min_dist']
    closest_pocket_id_ = df_pocket_.sort_values('min_dist').head(1).pocket_id.squeeze()
    if hasattr(closest_pocket_id_, '__len__'):
        return 0
    else:
        return closest_pocket_id_

# distance between every pocket & center-of-mass of the peptide
df_pockets['min_dist'] = closest_pocket_id(df_pockets, pepcom_x, pepcom_y, pepcom_z)
df_pockets[['struct_id', 'pocket_id', 'min_dist']]

,struct_id,pocket_id,min_dist
129896,P30101-F1,1,14.711930
129897,P30101-F1,2,37.412730
129898,P30101-F1,3,8.373239
129899,P30101-F1,4,62.017479
129900,P30101-F1,5,2.963212
129901,P30101-F1,6,37.493404
129902,P30101-F1,7,36.690638
129903,P30101-F1,8,51.921353
129904,P30101-F1,9,24.342558
129906,P30101-F1,11,59.575437


In [6]:
# Dictionary of pLDDT values (for structure)
d_ = read_af2_pLDDT('/cluster/project/beltrao/jjaenes/22.12_pocketomes/results/23.04_bindfunc/af2/P3/01/01/P30101-F1.pdb')
d_

OrderedDict([(1, 44.87),
             (2, 44.35),
             (3, 43.09),
             (4, 45.4),
             (5, 50.37),
             (6, 47.62),
             (7, 57.84),
             (8, 43.84),
             (9, 52.83),
             (10, 44.75),
             (11, 51.97),
             (12, 55.87),
             (13, 49.81),
             (14, 53.0),
             (15, 49.43),
             (16, 50.41),
             (17, 50.6),
             (18, 47.27),
             (19, 45.88),
             (20, 44.81),
             (21, 41.93),
             (22, 48.02),
             (23, 57.5),
             (24, 75.66),
             (25, 89.86),
             (26, 95.15),
             (27, 96.94),
             (28, 95.88),
             (29, 96.88),
             (30, 97.17),
             (31, 97.82),
             (32, 96.79),
             (33, 96.56),
             (34, 96.17),
             (35, 96.37),
             (36, 90.47),
             (37, 91.34),
             (38, 92.75),
             (39, 92.01),

In [7]:
# pLDDT values at peptide
pLDDT_at_peptide = [d_[pos] for pos in [219, 220, 221, 222, 223, 224, 225, 226]]
pLDDT_at_peptide

[98.07, 98.2, 98.01, 97.95, 96.53, 95.05, 91.39, 93.57]

In [8]:
# mean pLDDT of peptide
np.mean(pLDDT_at_peptide)

96.09625